# CO2 Emission Constraint Implementation & Validation

Validate the CO2 emission constraint added to REVOL-E-TION for depot electrification optimization.

## Objectives
1. Explain why CO2 constraints are needed for depot electrification planning
2. Document code changes made to REVOLETION
3. Demonstrate how constraints affect system design across 4 scenarios
4. Validate constraint correctness through compliance checks

## 1. Motivation & Context

The STRIDE thesis optimizes electric truck depot infrastructure over 25 years (2025-2050). Real-world depots face tightening CO2 emission constraints from regulatory mandates (EU Green Deal), corporate sustainability commitments, and policy scenarios modeling gradual decarbonization (e.g., 500 kg CO2 in 2025 → 100 kg in 2050).

REVOLETION's base framework only supports binary `res_only` mode—forcing 100% renewable grid imports. This can't model gradual emission reductions, stage-specific limits in multi-stage optimization, or flexible carbon budgets. The solution: add a `co2_max` parameter (in tonnes) to constrain total CO2 from grid imports, enabling policy scenario modeling, multi-stage optimization with tightening constraints, and investment-emission tradeoff analysis.

## 2. Implementation - Code Changes

Three files were modified to add CO2 emission constraints to REVOLETION:

### 2.1 Constraint Definition (`constraints.py`)

**File:** `revoletion/revoletion/constraints.py` (lines 296-329)

Added `limit_co2_emissions()` method to constrain total grid import emissions:

```python
def limit_co2_emissions(self, model):
    # Get co2_max from scenario (tonnes → kg)
    co2_max_tonnes = getattr(self.scenario, "co2_max", None)
    if co2_max_tonnes is None:
        return  # No constraint if not specified

    co2_max_kg = float(co2_max_tonnes) * 1000.0

    # Collect all grid import arcs and their emission factors
    import_arcs = []
    for grid in (b for b in self.scenario.blocks.values()
                 if isinstance(b, blocks.GridConnection)):
        for market in grid.subblocks.values():
            factor = getattr(market, "co2_spec_g2s", 0.4)  # kg CO2/kWh
            if factor and factor > 0:
                import_arcs.append((market.src, grid.bus, float(factor)))

    # Constraint: Σ(grid imports × emission factor) ≤ co2_max
    def _total_co2_kg():
        return sum(
            model.flow[src, bus, ts] * timestep_hours / 1000.0 * factor
            for (p, ts) in model.TIMEINDEX
            for (src, bus, factor) in import_arcs
        )

    blk.limit_co2_emissions = po.Constraint(expr=_total_co2_kg() <= co2_max_kg)
```

**Key Points:**
- Constraint applies to **simulation period** (e.g., 50 days), NOT project (25 years)
- Emission factor defaults to **0.4 kg CO2/kWh** (typical grid mix), user-configurable per GridMarket
- Automatically handles multi-grid scenarios

### 2.2 Diagnostic Output (`simulation.py`)

**File:** `revoletion/revoletion/simulation.py` (lines 225-248)

Added post-solve diagnostics to verify constraint compliance:

```python
if hasattr(self.model.CUSTOM_CONSTRAINTS, 'LIMIT_CO2'):
    c = self.model.CUSTOM_CONSTRAINTS.LIMIT_CO2.limit_co2_emissions

    print("=== CO2 constraint check ===")
    print("upper (kg):", value(c.upper))  # Limit
    print("body  (kg):", value(c.body))   # Actual emissions
    print("slack (kg):", value(c.upper - c.body))  # Headroom

    # Per-arc breakdown
    print("\nPer-arc CO2 breakdown:")
    for (src, bus, factor) in blk.import_arcs:
        arc_energy = sum(value(self.model.flow[src, bus, ts]) * timestep_hours / 1000.0
                        for (p, ts) in self.model.TIMEINDEX)
        arc_co2 = arc_energy * factor
        print(f"  {src.label} → {bus.label}: {arc_energy:.2f} kWh, {arc_co2:.2f} kg CO2")
```

Shows constraint body vs limit during optimization and identifies which grid sources contribute emissions.

### 2.3 CO2 Tracking in Results (`blocks.py`)

**File:** `revoletion/revoletion/blocks.py` (lines 957-962)

GridConnection block calculates CO2 emissions from grid imports:

```python
# Calculate CO2 emissions from grid imports (simulation period)
if hasattr(self, "flows") and len(self.flows) and "out" in self.flows:
    # flows['out'] = grid imports in W
    grid_import_kwh_sim = (self.flows["out"].clip(lower=0).sum() *
                           self.scenario.timestep_hours) / 1000.0
    self.co2_sim_kg = grid_import_kwh_sim * co2_factor
```

**Output:** CO2 emissions stored in summary CSV:  
- `grid,co2_sim_kg` - Simulation period emissions (what constraint tracks)  
- `grid,co2_prj_kg` - Project period emissions (25-year extrapolation)

### Summary: Files Modified

| File | Change | Lines |
|------|--------|-------|
| `constraints.py` | Added `limit_co2_emissions()` method | 33 |
| `simulation.py` | Added diagnostic output | 24 |
| `blocks.py` | Fixed `flows['in']` → `flows['out']` | 1 |

**Scenario Parameters:**
- `co2_max` - Maximum CO2 emissions in **tonnes** (None = unlimited)  
- `co2_spec_g2s` - Grid emission factor in **kg CO2/kWh** (default: 0.4, user-configurable per market)

---

## 3. Validation with 4 Scenarios

Validate constraint correctness across 4 CO2 limits: **100t, 10t, 1t, 0.1t**

### 3.1 Setup & Imports

In [1]:
import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from IPython.display import IFrame, display
import json
from datetime import datetime

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("✓ Imports successful")

✓ Imports successful


### 3.2 Define Paths

In [2]:
# Project paths
REPO_ROOT = Path("/Users/arnoclaude/Documents/TUM/Thesis/STRIDE")
REVOLETION_DIR = REPO_ROOT / "revoletion"
EXAMPLE_DIR = REVOLETION_DIR / "example"
RESULTS_DIR = REVOLETION_DIR / "results"

# Notebook output directory
NOTEBOOK_OUTPUT = REPO_ROOT / "notebooks" / "01_test_co2_constraint" / "output"
NOTEBOOK_OUTPUT.mkdir(exist_ok=True, parents=True)

# Scenario files - using pre-configured 4-scenario CSV
SCENARIO_FILE = EXAMPLE_DIR / "scenarios_example_4_bev_mg_co2_constraint.csv"
SETTINGS_FILE = EXAMPLE_DIR / "settings.csv"

print(f"✓ Repository root: {REPO_ROOT}")
print(f"✓ Scenario file: {SCENARIO_FILE}")
print(f"✓ Output directory: {NOTEBOOK_OUTPUT}")

✓ Repository root: /Users/arnoclaude/Documents/TUM/Thesis/STRIDE
✓ Scenario file: /Users/arnoclaude/Documents/TUM/Thesis/STRIDE/revoletion/example/scenarios_example_4_bev_mg_co2_constraint.csv
✓ Output directory: /Users/arnoclaude/Documents/TUM/Thesis/STRIDE/notebooks/01_test_co2_constraint/output


### 3.3 Run Optimization

Run REVOLETION with 4 scenarios (**bev_mg_1** through **bev_mg_4**) in parallel:

In [3]:
# Run optimization (all 4 scenarios processed in parallel)
print("Running REVOLETION optimization...")
print(f"Scenario file: {SCENARIO_FILE.name}")
print("Scenarios: bev_mg_1 (100t), bev_mg_2 (10t), bev_mg_3 (1t), bev_mg_4 (0.1t)\n")

result = subprocess.run([
    'python3', '-m', 'revoletion.main',
    '--settings', str(SETTINGS_FILE),
    '--scenario', str(SCENARIO_FILE)
], cwd=str(REVOLETION_DIR), capture_output=True, text=True, timeout=600)

# Check if successful
if result.returncode == 0:
    print("✓ Optimization completed successfully")
else:
    print(f"✗ Optimization failed with return code {result.returncode}")
    print(result.stderr)

Running REVOLETION optimization...
Scenario file: scenarios_example_4_bev_mg_co2_constraint.csv
Scenarios: bev_mg_1 (100t), bev_mg_2 (10t), bev_mg_3 (1t), bev_mg_4 (0.1t)

✓ Optimization completed successfully


### 3.4 Load Results

Find latest result directory and parse the **single summary CSV** containing all 4 scenarios:

In [4]:
# Find latest result directory (contains all 4 scenarios)
result_dirs = sorted([p for p in RESULTS_DIR.glob("*4_bev_mg_co2_constraint") if p.is_dir()],
                    key=lambda p: p.stat().st_mtime, reverse=True)

if not result_dirs:
    raise FileNotFoundError("No result directories found matching '*4_bev_mg_co2_constraint'")

result_dir = result_dirs[0]  # Latest run
print(f"✓ Result directory: {result_dir.name}")

# Load summary CSV (all 4 scenarios as columns)
summary_csv = list(result_dir.glob("*_summary.csv"))[0]
df_summary = pd.read_csv(summary_csv)

print(f"✓ Loaded summary CSV: {summary_csv.name}")
print(f"  Columns: {list(df_summary.columns)}")
print(f"  Rows: {len(df_summary)}")

✓ Result directory: 251129_115844_scenarios_example_4_bev_mg_co2_constraint
✓ Loaded summary CSV: 251129_115844_scenarios_example_4_bev_mg_co2_constraint_summary.csv
  Columns: ['Block', 'Key', 'bev_mg_1', 'bev_mg_2', 'bev_mg_3', 'bev_mg_4']
  Rows: 1643


### 3.5 Extract Metrics

Parse metrics for all 4 scenarios from the summary CSV:

In [5]:
def get_metric(block, key):
    """Extract metric from summary CSV for all 4 scenarios."""
    mask = (df_summary['Block'] == block) & (df_summary['Key'] == key)
    if mask.any():
        row = df_summary.loc[mask].iloc[0]
        return {
            'bev_mg_1': row['bev_mg_1'],
            'bev_mg_2': row['bev_mg_2'],
            'bev_mg_3': row['bev_mg_3'],
            'bev_mg_4': row['bev_mg_4']
        }
    return None

# Extract key metrics
metrics = {
    'co2_max_tonnes': get_metric('scenario', 'co2_max'),
    'co2_actual_kg': get_metric('grid', 'co2_sim_kg'),
    'pv_size_w': get_metric('pv', 'size_total'),
    'ess_size_wh': get_metric('ess', 'size_total'),
    'capex_usd': get_metric('scenario', 'capex_prj'),
    'opex_usd': get_metric('scenario', 'opex_prj'),
    'npv_usd': get_metric('scenario', 'npv'),
    'lcoe': get_metric('scenario', 'lcoe_total')
}

# Calculate grid imports from CO2 emissions (co2 = grid_import × emission_factor)
# Using default emission factor of 0.4 kg CO2/kWh
emission_factor = 0.4
metrics['grid_import_kwh'] = {}
for scenario in ['bev_mg_1', 'bev_mg_2', 'bev_mg_3', 'bev_mg_4']:
    co2_kg = float(metrics['co2_actual_kg'][scenario])
    metrics['grid_import_kwh'][scenario] = co2_kg / emission_factor

# Convert to DataFrame for easier manipulation
df_metrics = pd.DataFrame(metrics).T
df_metrics.columns = ['bev_mg_1', 'bev_mg_2', 'bev_mg_3', 'bev_mg_4']

print("✓ Extracted metrics for all 4 scenarios")
print(f"✓ Grid imports derived from CO2 emissions (factor={emission_factor} kg/kWh)")
df_metrics

✓ Extracted metrics for all 4 scenarios
✓ Grid imports derived from CO2 emissions (factor=0.4 kg/kWh)


,bev_mg_1,bev_mg_2,bev_mg_3,bev_mg_4
co2_max_tonnes,100,10,1,0.1
co2_actual_kg,3408.404301444107,3386.573704382914,999.999999999999,99.99999999999989
pv_size_w,0.0,0.0,108093.17673395741,180079.51902061285
ess_size_wh,10000.0,10000.0,49136.254942305386,301734.3651821002
capex_usd,182513.4610460393,182477.28173500168,252504.58414412322,351429.69101231155
opex_usd,141395.28631319592,135398.15308694652,91443.85929657753,66978.63539943525
npv_usd,-241484.41019483397,-239272.06094929914,-283399.70005575125,-374168.08893211925
lcoe,0.0012538710640764574,0.0012423966581812447,0.0012226003281031203,0.0011718084422180023
grid_import_kwh,8521.010754,8466.434261,2500.0,250.0


### 3.6 Comparison Table

In [6]:
# Create formatted comparison table
comparison_data = []

for scenario in ['bev_mg_1', 'bev_mg_2', 'bev_mg_3', 'bev_mg_4']:
    # Convert to float to handle string values from CSV
    co2_limit_tonnes = float(metrics['co2_max_tonnes'][scenario]) if pd.notna(metrics['co2_max_tonnes'][scenario]) else None
    co2_limit_kg = co2_limit_tonnes * 1000 if co2_limit_tonnes else None
    
    comparison_data.append({
        'Scenario': scenario,
        'CO2 Limit (kg)': f"{co2_limit_kg:,.0f}" if co2_limit_kg else "Unlimited",
        'Actual CO2 (kg)': f"{float(metrics['co2_actual_kg'][scenario]):,.1f}",
        'PV (kW)': f"{float(metrics['pv_size_w'][scenario])/1000:,.1f}",
        'Battery (kWh)': f"{float(metrics['ess_size_wh'][scenario])/1000:,.1f}",
        'Grid Import (kWh)': f"{metrics['grid_import_kwh'][scenario]:,.1f}",
        'CAPEX ($)': f"{float(metrics['capex_usd'][scenario]):,.0f}",
        'OPEX ($)': f"{float(metrics['opex_usd'][scenario]):,.0f}",
        'NPV ($)': f"{float(metrics['npv_usd'][scenario]):,.0f}"
    })

df_comparison = pd.DataFrame(comparison_data)

print("SCENARIO COMPARISON - CO2 CONSTRAINT VALIDATION")
print(df_comparison.to_string(index=False))

# Save to CSV
df_comparison.to_csv(NOTEBOOK_OUTPUT / 'comparison_table.csv', index=False)
print(f"\n✓ Saved to: {NOTEBOOK_OUTPUT / 'comparison_table.csv'}")

SCENARIO COMPARISON - CO2 CONSTRAINT VALIDATION
Scenario CO2 Limit (kg) Actual CO2 (kg) PV (kW) Battery (kWh) Grid Import (kWh) CAPEX ($) OPEX ($)  NPV ($)
bev_mg_1        100,000         3,408.4     0.0          10.0           8,521.0   182,513  141,395 -241,484
bev_mg_2         10,000         3,386.6     0.0          10.0           8,466.4   182,477  135,398 -239,272
bev_mg_3          1,000         1,000.0   108.1          49.1           2,500.0   252,505   91,444 -283,400
bev_mg_4            100           100.0   180.1         301.7             250.0   351,430   66,979 -374,168

✓ Saved to: /Users/arnoclaude/Documents/TUM/Thesis/STRIDE/notebooks/01_test_co2_constraint/output/comparison_table.csv


### 3.7 Constraint Validation Checks

In [7]:
print("\nCONSTRAINT COMPLIANCE VALIDATION")
print("="*80)
print("Note: CO2 constraint applies to SIMULATION period (50 days)\n")

tolerance_kg = 1.0  # Allow 1 kg numerical tolerance
all_passed = True

for scenario in ['bev_mg_1', 'bev_mg_2', 'bev_mg_3', 'bev_mg_4']:
    co2_limit_tonnes = metrics['co2_max_tonnes'][scenario]
    co2_actual_kg = float(metrics['co2_actual_kg'][scenario])
    
    if pd.notna(co2_limit_tonnes):
        co2_limit_kg = float(co2_limit_tonnes) * 1000
        slack_kg = co2_limit_kg - co2_actual_kg
        
        if co2_actual_kg <= co2_limit_kg + tolerance_kg:
            status = "✓ PASS"
        else:
            status = "✗ FAIL"
            all_passed = False
        
        print(f"{status} | {scenario}: {co2_actual_kg:>8.1f} kg ≤ {co2_limit_kg:>10,.0f} kg (slack: {slack_kg:>8.1f} kg)")
    else:
        print(f"  INFO | {scenario}: {co2_actual_kg:>8.1f} kg (unlimited)")

print("="*80)
if all_passed:
    print("\n✅ ALL CONSTRAINTS SATISFIED - Implementation validated!")
else:
    print("\n⚠️  CONSTRAINT VIOLATIONS DETECTED - Review implementation")


CONSTRAINT COMPLIANCE VALIDATION
Note: CO2 constraint applies to SIMULATION period (50 days)

✓ PASS | bev_mg_1:   3408.4 kg ≤    100,000 kg (slack:  96591.6 kg)
✓ PASS | bev_mg_2:   3386.6 kg ≤     10,000 kg (slack:   6613.4 kg)
✓ PASS | bev_mg_3:   1000.0 kg ≤      1,000 kg (slack:      0.0 kg)
✓ PASS | bev_mg_4:    100.0 kg ≤        100 kg (slack:      0.0 kg)

✅ ALL CONSTRAINTS SATISFIED - Implementation validated!


---

## 4. Summary Statistics

In [8]:
# Prepare data for summary export
scenarios = ['bev_mg_1', 'bev_mg_2', 'bev_mg_3', 'bev_mg_4']
co2_limits = [float(metrics['co2_max_tonnes'][s]) * 1000 for s in scenarios]  # kg
pv_kw = [float(metrics['pv_size_w'][s]) / 1000 for s in scenarios]
ess_kwh = [float(metrics['ess_size_wh'][s]) / 1000 for s in scenarios]
co2_actual = [float(metrics['co2_actual_kg'][s]) for s in scenarios]
npv_k = [float(metrics['npv_usd'][s]) / 1000 for s in scenarios]

# Calculate summary statistics
pv_increase_pct = ((pv_kw[-1] / pv_kw[0]) - 1) * 100 if pv_kw[0] > 0 else float('inf')
ess_increase_pct = ((ess_kwh[-1] / ess_kwh[0]) - 1) * 100
co2_reduction_pct = ((co2_actual[0] - co2_actual[-1]) / co2_actual[0]) * 100
npv_impact = npv_k[-1] - npv_k[0]

summary = {
    'test_date': datetime.now().isoformat(),
    'result_directory': str(result_dir.name),
    'scenarios_tested': 4,
    'all_constraints_passed': all_passed,
    'baseline_scenario': 'bev_mg_1',
    'tightest_scenario': 'bev_mg_4',
    'metrics': {
        'pv_increase_pct': round(pv_increase_pct, 1),
        'ess_increase_pct': round(ess_increase_pct, 1),
        'co2_reduction_pct': round(co2_reduction_pct, 1),
        'npv_impact_k_usd': round(npv_impact, 1)
    },
    'scenarios': {
        s: {
            'co2_limit_kg': float(co2_limits[i]),
            'co2_actual_kg': round(co2_actual[i], 1),
            'pv_kw': round(pv_kw[i], 1),
            'ess_kwh': round(ess_kwh[i], 1),
            'npv_k_usd': round(npv_k[i], 1)
        } for i, s in enumerate(scenarios)
    }
}

# Save to JSON
with open(NOTEBOOK_OUTPUT / 'summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print(f"✓ Saved summary statistics to: {NOTEBOOK_OUTPUT / 'summary.json'}")

✓ Saved summary statistics to: /Users/arnoclaude/Documents/TUM/Thesis/STRIDE/notebooks/01_test_co2_constraint/output/summary.json
